In [ ]:
!pip install -q demucs fastapi uvicorn python-multipart pyngrok
!apt-get install -y ffmpeg

In [ ]:
%%writefile app.py
import os
import uuid
import shutil
import subprocess
from pathlib import Path
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
from fastapi.staticfiles import StaticFiles

# ==============================
# CONFIG
# ==============================

MAX_DURATION_SECONDS = 300
ALLOWED_EXTENSIONS = [".wav", ".mp3"]
SEGMENT_SIZES = [7, 6, 5]
OVERLAP = 0.1
MODEL_NAME = "htdemucs"

BASE_DIR = "content"
UPLOAD_DIR = "uploads"
OUTPUT_DIR = "outputs"

os.makedirs(UPLOAD_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

app = FastAPI()

# Serve output stems publicly
app.mount("/outputs", StaticFiles(directory=OUTPUT_DIR), name="outputs")


# ==============================
# UTILS
# ==============================

def get_audio_duration(filepath: str):
    cmd = [
        "ffprobe",
        "-v", "error",
        "-show_entries", "format=duration",
        "-of", "default=noprint_wrappers=1:nokey=1",
        filepath
    ]
    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    if result.returncode != 0:
        return None

    try:
        return float(result.stdout.strip())
    except:
        return None


def run_demucs(input_path: str, job_id: str):
    output_root = f"{OUTPUT_DIR}/{job_id}"
    os.makedirs(output_root, exist_ok=True)

    for seg in SEGMENT_SIZES:
        cmd = [
            "demucs",
            "-n", MODEL_NAME,
            "--segment", str(seg),
            "--overlap", str(OVERLAP),
            "-o", output_root,
            input_path
        ]

        result = subprocess.run(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )

        if result.returncode == 0:
            return True, output_root

        if "CUDA out of memory" in result.stderr:
            continue

        return False, result.stderr

    return False, "GPU out of memory even after fallback."


# ==============================
# API
# ==============================

@app.post("/separate")
async def separate(file: UploadFile = File(...)):

    ext = Path(file.filename).suffix.lower()

    if ext not in ALLOWED_EXTENSIONS:
        return JSONResponse(
            status_code=400,
            content={"status": "error", "message": "Unsupported file type"}
        )

    job_id = str(uuid.uuid4())
    filename_stem = Path(file.filename).stem
    input_path = f"{UPLOAD_DIR}/{job_id}{ext}"

    # Save upload
    with open(input_path, "wb") as buffer:
        shutil.copyfileobj(file.file, buffer)

    # Validate duration
    duration = get_audio_duration(input_path)
    if duration is None:
        return {"status": "error", "message": "Could not read audio duration"}

    if duration > MAX_DURATION_SECONDS:
        return {"status": "error", "message": "Audio exceeds 5 minute limit"}

    # Run separation
    success, result = run_demucs(input_path, job_id)
    if not success:
        return {"status": "error", "message": result}

    # Build public URLs
    stem_base = f"/outputs/{job_id}/{MODEL_NAME}/{job_id}"

    return {
        "status": "success",
        "job_id": job_id,
        "stems": {
            "vocals": f"{stem_base}/vocals.wav",
            "drums": f"{stem_base}/drums.wav",
            "bass": f"{stem_base}/bass.wav",
            "other": f"{stem_base}/other.wav",
        }
    }

In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token("XXXX")

In [ ]:
from pyngrok import ngrok
import subprocess
import time

# kill everything
!pkill -f ngrok
!pkill -f uvicorn

time.sleep(2)

# open ONE tunnel
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

# start server
subprocess.Popen(["uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"])

In [ ]:
!pkill -f ngrok
!pkill -f uvicorn